<a href="https://colab.research.google.com/github/BryanFerreiraH/IA/blob/master/preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importar daos de Google Drive con Google Colab
from google.colab import drive

# Activa la autentificacion para acceder a datos
drive.mount('/content/drive')

#Nos muestra el directorio de nuestro proyecto
!ls "/content/drive/Team Drives/IA Proyecto Final"

In [0]:
#Rutas de .mat con datos ordenador
r_train = './dmat/train_32x32.mat'
r_test = './dmat/test_32x32.mat'
#r_extra = './dmat/test_32x32.mat'

In [0]:
#Rutas de .mat con datos en la nube
r_train = '/content/drive/Team Drives/IA Proyecto Final/dataset/train_32x32.mat'
r_test = '/content/drive/Team Drives/IA Proyecto Final/dataset/test_32x32.mat'
r_extra = '/content/drive/Team Drives/IA Proyecto Final/dataset/extra_32x32.mat'

In [0]:
import numpy as np #Manejo de matrices 
import scipy.io as sio #Leer datos de archivo original .mat
import matplotlib.pyplot as plt #Mostrar datos
from sklearn.preprocessing import OneHotEncoder as hot_enc #Para trabajar con los datos en CNN
import h5py #Guardar en h5 para luego procesarlos
image_ind = 6982 #Acceso rapido para imprimir

In [0]:
#Cargar datos de diccionarios
train_data = sio.loadmat(r_train)
test_data = sio.loadmat(r_test)
extra_data = sio.loadmat(r_extra)

#Datos ENTRENAMIENTO
x_train = train_data['X']
y_train = train_data['y']
#Datos TEST
x_test = test_data['X']
y_test = test_data['y']
#Datos EXTRA
#x_extra = extra_data['X']
y_extra = extra_data['y']
del extra_data
y_extra = y_extra[:200000,:]

In [5]:
y_extra.shape

(200000, 1)

In [0]:
#Se realiza un transpose al principio del tratamiento debido a la forma en la que
#una CNN recibe los datos ya que de manera predeterminada espera los datos con
#formato datos,ancho,alto,canales.
x_train = x_train.transpose((3,0,1,2))
x_test = x_test.transpose((3,0,1,2))
#x_extra = x_extra.transpose((3,0,1,2))

print("x_train tiene y y_train tienen ", x_train.shape[0], " datos")

#Contamos la aparición de cada elemento en un numpy
unique, counts = np.unique(y_train, return_counts=True)
print("Los datos en y_train estan distribuidos:")
print(dict(zip(unique, counts)))
#del unique,counts,train_data

In [0]:
#Seaborn histograma revisar tiempo de ejecucion
#sns.distplot(y_test, bins=20, kde=False, rug=True);

In [0]:
#Grafica que muestra la distribucion de numeros
graf1, (training, test) = plt.subplots(1, 2, sharex=True)

training.hist(y_train, color= '#9b9b9b', bins=20)
training.set_title("Datos entrenamiento")

test.hist(y_test, color='#009C8C', bins=20)
test.set_title("Datos de prueba")

graf1.tight_layout()

In [0]:
#Usamos np.unique para ver los valores de y_train, notemos que hay un
#error en la data ya que el 10 es en realidad un 0.
print("Numeros en y_train: ",np.unique(y_train), "<- Error con 10")
#def unicos(lista):
    #conjunto = set(lista)
    #lista = list(conjunto)
    #print(lista)  #NO FUNCIONA PQ ES unhashable ya es numpyarray
 
def arreglar_lista(lista_y):
    lista_y[lista_y == 10] = 0 #Modificamos el 10 por 0

arreglar_lista(y_train)
arreglar_lista(y_test)
arreglar_lista(y_extra)

In [0]:
#print("Numeros actualizados", np.unique(y_train), "<- Correcto")
graf2, (training, test) = plt.subplots(1, 2, sharex=True)

training.hist(y_train, color= '#9b9b9b', bins=20)
training.set_title("Datos entrenamiento")

test.hist(y_test, color='#009C8C', bins=20)
test.set_title("Datos de prueba")

graf2.tight_layout()

In [0]:
# Y = 0.2990R + 0.5870G + 0.1140B --> Formula
def rgb_gray(datos):
    return np.dot(datos, [0.2990, 0.5870, 0.1140])

In [0]:
#Funcion para estructura de print
#Ejemplo de como print
#pimg(x_train), si image_ind es 1 buscara el de indice 1 en x_train y lo print
#Nuestra variable independiente esta estructurada[tamaño,ancho,altura,canales]
def pimg (data):
    plt.imshow(data[image_ind,:,:,:]) ##image_ind es un indice para buscar definido arriba
    plt.show()
    print("Numero de imagen",y_train[image_ind])

In [0]:
#Cambiar de rgb a gray , con np.dot se eliminan los canales.
x_train_gris = rgb_gray(x_train)
print(x_train_gris.shape)

x_test_gris = rgb_gray(x_test)
x_test_gris.shape

In [0]:
#Ejemplo de como imprimir es gris
#plt.imshow(x_train_gris[image_ind,:,:], cmap='gray')
#plt.show()

In [0]:
#Funcion para normalizacion de datos
def normalizar(datos):
    media = np.mean(datos, axis=0)
    #Desviacion estandar
    desv = np.std(datos, axis=0)
    # Datos normalizados
    return (datos - media) / desv

x_train_gris_n = normalizar(x_train_gris)
x_test_gris_n = normalizar(x_test_gris)
#print(x_train_gris_n[1,1,1]) ##Borrar, es una prueba de valor
#print(x_train_gris_n[-1,-1,-1]) ##Borrar, es una prueba de valor

In [0]:
enc = hot_enc().fit(y_train.reshape(-1, 1))

#Explicación de esto... https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f
#Explicar en presentacion, se pasan de 0:9 a algo similar a matrix de confusion
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
y_extra = enc.transform(y_extra.reshape(-1, 1)).toarray()

print("y train", y_train.shape)
print("y test", y_test.shape)
print("y extra", y_extra.shape)

In [0]:
#Crea un archivo para guardar los datos
h5f = h5py.File('datos.h5', 'w')

#Entrenamiento
h5f.create_dataset('x_train', data=x_train_gris_n)
h5f.create_dataset('y_train', data=y_train)
#Test
h5f.create_dataset('x_test', data=x_test_gris_n)
h5f.create_dataset('y_test', data=y_test)
#Extra
h5f.create_dataset('y_extra', data=y_extra)
#Cerrar y guardar
h5f.close()